In [1]:
### import libraries
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from matplotlib import gridspec
from scipy.optimize import curve_fit
from scipy import signal

from sklearn import preprocessing
from sklearn.metrics import r2_score
%matplotlib inline

## constant parameters , check from tracking docs in shared folder, D, phi_0, epsilon_r will change based on metals 
## and dielctric layer involved

D_0 = 50  #gap size (angstrome) ## from ALD cycles

phi_0_0 = 4.9  #work function (eV)

epsilon_r_0_0 = 3.8  #dielectric constant (ratio)

T0 = 298 #(K)    

def x_1(V_g,D,phi_0,epsilon_r_0): 

    return 3/(epsilon_r_0*phi_0) 



def x_2(V_g,D,phi_0,epsilon_r_0):

    return np.where(V_g < phi_0, D*(1-46/(6*phi_0*epsilon_r_0*D + 20 - 4*V_g*epsilon_r_0*D)) + 3/(epsilon_r_0*phi_0) , 
                    (phi_0*epsilon_r_0*D - 14)/(epsilon_r_0*V_g) )


def SimmonsFit(V_g,I_0,C_leakage,D,phi_0,epsilon_r_0): #three parameters dielectric constant ,gap size, work function , bounds will be given
    
    x1 = x_1(V_g,D,phi_0,epsilon_r_0)

    x2 = x_2(V_g,D,phi_0,epsilon_r_0)

    dx = x2 - x1
    
    phi_L = phi_0 - (V_g/(2*D))*(x1+x2) - (5.75/(2*epsilon_r_0*dx))*np.log((x2*(D-x1))/(x1*(D-x2)))
    
    return I_0 + V_g*C_leakage + (1e18/4)*((6.2e10/(dx**2))*(phi_L*np.exp(-1.025*dx*(phi_L**.5)) - 
                                              (phi_L + V_g)*np.exp(-1.025*dx*((phi_L + V_g)**.5)))*
                                     (1 + ((3e-9*(dx**2)*(T0**2))/phi_L)))      #nanoampere   #

   

In [2]:

die="W014"  ## name of wafer
#die = "W009"
#die = "W005"
#die = "W013"

url = 'http://10.212.29.250:3310/DataDownload/DataSetShow?die=' + die    ## Brian's RAX server
response = requests.get(url)
items=response.json()
breaks = [x for x in items] 

In [4]:
## constant parameters , check from tracking docs in shared folder, D, phi_0, epsilon_r will change based on metals 
## and dielctric layer involved


##For csv data storage
r2_all= [] 
cap_offset =[] 
leakage_cond = []
gap_size =[]
work_func =[]
die_const = []
chip_name_all =[]
for b in breaks:
    chip_name = b['Title']                             ## read title of chip
    chip = b['Searchable']
    
    #if b['DataJson'] is None:
    #    print('NoneType object')   ## nonetype error override
        
    if b['DataJson'] is not None:
        
        current = np.array(b['DataJson']['Current'])*1e9   ## read current in A, convert to nA
        voltage = np.abs(np.array(b['DataJson']['Voltage']))       ## read bias in Volts
        if np.abs(np.min(voltage)-np.max(voltage)) >=4 :
            
            print(len(voltage))                         ## name and condition of chip
            ## slope of linear I-V portion to get leakage current from conductance

            voltage_roi = np.append(voltage[5:len(voltage)-10],voltage[len(voltage)-2:])
            current_roi = np.append(current[5:len(voltage)-10],current[len(voltage)-2:])
            
            ## not avoiding the tunneling exponential and not avoiding the charging curve.  May need some logic to figure out 
            ## which part is linear
            C_leakage_0, I_0_0 = np.polyfit(voltage[-2:], current[-2:], 1)  
            
            print(voltage[-2:])
            print(current[-2:])
            break
            
            print('C_leakage',C_leakage_0)
            print('I_0_0',I_0_0)
            
            # Fit the current data
            
            p0=[I_0_0,C_leakage_0,D_0, phi_0_0, epsilon_r_0_0] 
            
            # Parameters are I_0_0, C_leakage, D,phi_0,epsilon_r_0
            
            
            ## not avoiding the charging curve here.  Need to clip the data range to avoid the charging curve, and then clip the 
            ## tunneling part to avoid really high currents where simmon's fails
            pars, cov = curve_fit(f=SimmonsFit, xdata = voltage_roi,ydata = current_roi, p0 = p0,
                                  bounds=((-np.inf,-np.inf,10, 3.5,3), (np.inf,np.inf,80, 6.5, 5)),
                                  method ='trf',maxfev=50000) #, 'lm',‘trf’, ‘dogbox’)
            
            # Get the standard deviations of the parameters (square roots of the # diagonal of the covariance)
            
            #stdevs = np.sqrt(np.diag(cov))
            
            # Calculate the residuals
            #res = current - SimmonsFit(current, *pars)

            simmons_current = SimmonsFit(voltage_roi,*pars)                                                  
    
            ### Coefficient of determination to show how close the fitted current is to the experimental true values
        
            r2 = r2_score(current_roi, simmons_current)

            
            if 0.7 < r2 < 1:
                
                print(chip)
                chip_name_all.append(chip)
                r2_all.append(r2)
                cap_offset.append(pars[0]) 
                leakage_cond.append(pars[1])
                gap_size.append(pars[2])
                work_func.append(pars[3])
                die_const.append(pars[4])

                chip_dict = {'Goodness of fit' : r2 ,'Capacitance offset': pars[0],'Leakage Resistance (nS)': (1/pars[1]),
                             'Gap Size (Angstrome) ': pars[2], 'Work Function (eV)': pars[3], 
                             'Dielectric Constant' : pars[4]}
                
                print(chip_dict)
                
                ### Plot all figures
                
                fig = plt.figure(figsize=(6,4))
                
                ### Plot the fitted current data as an overlay on the scatter data.
                ### Lookout for exceptions where bias values are highly negative instead of positive.
                plt.plot(voltage, current,linestyle='-', linewidth=3,color='blue')
                #plt.plot(voltage_roi,current_roi, linestyle='-.', linewidth=3,color='orange')
                plt.plot(voltage_roi,simmons_current ,linestyle='--', linewidth=3, color='green')
                plt.plot(voltage[-2:],current[-2:],'ro',markersize=5)
                plt.xlabel("Potential (V)", fontsize = 14)
                plt.ylabel("Current (nA)", fontsize = 14)
                plt.title(chip)
                plt.tight_layout()
                #plt.savefig(f'{chip}.png')
                plt.show()
                
### Excel file of params

df = pd.DataFrame({'Chip Name': chip_name_all,'Goodness of fit' : r2_all ,'Capacitance offset': cap_offset,'Leakage Conductance (nS)': leakage_cond,
                   'Gap Size (Angstrome) ': gap_size, 'Work Function (eV)': work_func, 'Dielectric Constant' : die_const}, index = None)
#df.to_csv(f'Simmons_fit_{die}.csv', header=True)


201
[8.000689 0.      ]
[0.02944469 0.        ]


In [4]:
#  # Fowler-Nordheim equation

import scipy as sp
import scipy.constants
import numpy as np

# CONSTANTS #
kb = sp.constants.value("Boltzmann constant in eV/K") # unit: eV/K
qe = sp.constants.value("elementary charge") # unit: C
me = sp.constants.value("electron mass")/qe #unit: eV*s^2/m^2
hp = sp.constants.value("Planck constant in eV s") #unit: eV*s
hbar = hp/(2*sp.pi) #unit: eV*s
eps0 = sp.constants.value("electric constant")*qe #unit: C^2/(eV*m)
a_rld = 4*sp.pi*me*kb**2*qe/hp**3 #unit: C/(K^2*s*m^2)

# FN constants:
a_fn = qe**3/(8*sp.pi*hp) #unit: C^3/(eV*s)
b_fn = 8*sp.pi*np.sqrt(2*me)/(3*qe*hp)  #unit: 1/(C*eV^0.5*m)

# Units:
# current density [A/m^2]: j(F[GV/m], phi[eV])
# inputs: field F [GV/m]
#         work function phi [eV]
def FNfit(F,I_0,C_leakage, phi):

    # Convert field from [GV/m] to [eV/C*m] to match other units
    F = 1e9/qe*F  

    fb = 4*sp.pi*eps0*phi**2/qe**3
    nu = 1-F/fb+1/6*F/fb*np.log(F/fb)
    tau = 1+1/9*F/fb*(1-1/2*np.log(F/fb))

    return I_0 + F*C_leakage + 1e-9*a_fn*F**2/(phi*tau**2)*np.exp(-nu*b_fn*phi**(3/2)/F)

# Units:
# current density [A/m^2]: j(F[GV/m], phi[eV])
# inputs: field F [GV/m]
#         temperature [K]
#         work function phi [eV]
#def j_temp(F, T, phi):
#    if T <= 0:
#        T = 1
#    F_conv = 1e9/qe*F
#    dt = 2*F_conv/(3*b_fn*np.sqrt(phi))
#    theta = np.pi*kb*T/dt/np.sin(np.pi*kb*T/dt)
#    return theta*j(F, phi)

In [5]:
die="W014"  ## name of wafer
#die = "W009"
#die = "W005"
#die = "W013"

url = 'http://10.212.29.250:3310/DataDownload/DataSetShow?die=' + die    ## Brian's RAX server
response = requests.get(url)
items=response.json()
breaks = [x for x in items] 

##For csv data storage
r2_all= [] 
work_func = []
chip_name_all =[]
for b in breaks:
    chip_name = b['Title']                             ## read title of chip
    chip = b['Searchable']
    
    #if b['DataJson'] is None:
    #    print('NoneType object')   ## nonetype error override
        
    if b['DataJson'] is not None:
        
        current = np.array(b['DataJson']['Current'])*1e9   ## read current in A, convert to nA
        voltage = np.abs(np.array(b['DataJson']['Voltage']))       ## read bias in Volts
        if np.abs(np.min(voltage)-np.max(voltage)) >=4 :
            
            #print(b['Searchable'])                         ## name and condition of chip
            
            ## slope of linear I-V portion to get leakage current from conductance
            
            voltage_roi = voltage[3:len(voltage)]
            current_roi = current[3:len(voltage)]
            
            
            C_leakage_0, I_0_0 = np.polyfit(voltage[:-2], current[:-2], 1)  
            print('C_leakage',C_leakage_0)
            print('I_0_0',I_0_0)
            
            # Fit the current data
            
            p0=[I_0_0,C_leakage_0,phi_0_0] 
            
            # Parameters are D,phi_0,epsilon_r_0
            
            pars, cov = curve_fit(f=FNfit, xdata = voltage_roi,ydata = current_roi, p0 = p0,
                                  method ='lm',maxfev=5000) #, 'lm',‘trf’, ‘dogbox’)
            
            # Get the standard deviations of the parameters (square roots of the # diagonal of the covariance)
            stdevs = np.sqrt(np.diag(cov))
            #print(pars)
            
            # Calculate the residuals
            #res = current - SimmonsFit(current, *pars)

            fn_current = (FNfit(voltage_roi,*pars))                                              
            print(fn_current)
            
            ### Coefficient of determination to show how close the fitted current is to the experimental true values
            r2 = r2_score(current_roi, fn_current)

            
            if 0.7 < r2 < 1:
                
                print(chip)
                chip_name_all.append(chip)
                r2_all.append(r2)
                
                work_func.append(pars[0])

                chip_dict = {'Goodness of fit' : r2 ,'Work Function (eV)': pars[0]} 
                           
                
                print(chip_dict)
                ### plot all figures
                fig = plt.figure(figsize=(6,4))
                
                ### Plot the fitted current data as an overlay on the scatter data.
                ### Lookout for exceptions where bias values are highly negative instead of positive.
                
                plt.plot(voltage_roi,current_roi*1e9, linestyle='-', linewidth=3)
                plt.plot(voltage_roi,fn_current*1e9, linestyle='--', linewidth=3, color='green')
                plt.xlabel("Potential (V)", fontsize = 14)
                plt.ylabel("Current (nA)", fontsize = 14)
                plt.title(chip)
                plt.tight_layout()
                plt.show()
                

C_leakage 0.00030614575355059927
I_0_0 0.0273102456939118
[2.30087118e+23 3.07370463e+23 3.84011583e+23 4.60538628e+23
 5.37826750e+23 6.14408444e+23 6.91117780e+23 7.68355647e+23
 8.45033265e+23 9.21624131e+23 9.98934800e+23 1.07553942e+24
 1.15210755e+24 1.22936816e+24 1.30604119e+24 1.38270486e+24
 1.45984718e+24 1.53654276e+24 1.61314739e+24 1.68982940e+24
 1.76702179e+24 1.84362641e+24 1.92034072e+24 1.99759100e+24
 2.07418779e+24 2.15081516e+24 2.22808837e+24 2.30477497e+24
 2.38135647e+24 2.45861823e+24 2.53520928e+24 2.61185002e+24
 2.68902388e+24 2.76570665e+24 2.84234739e+24 2.91954800e+24
 2.99613905e+24 3.07281228e+24 3.15010078e+24 3.22677400e+24
 3.30334213e+24 3.37997331e+24 3.45728857e+24 3.53390256e+24
 3.61055667e+24 3.68764836e+24 3.76428146e+24 3.84105022e+24
 3.91817821e+24 3.99481895e+24 4.07143676e+24 4.14867558e+24
 4.22527428e+24 4.30187107e+24 4.37912709e+24 4.45574108e+24
 4.53230348e+24 4.60961873e+24 4.68620023e+24 4.76288683e+24
 4.84004349e+24 4.91672053e

C:\Users\smukhe33\anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
C:\Users\smukhe33\anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in multiply
C:\Users\smukhe33\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
C:\Users\smukhe33\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply
C:\Users\smukhe33\anaconda3\lib\site-packages\scipy\optimize\minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').